# MIS Kinect IMG Capture Module

## Video Setting

In [1]:
import os
import cv2
import numpy as np
import pykinect_azure as pykinect
from pykinect_azure.k4a import calibration
from pykinect_azure.k4a import _k4a
import time

# file name setting
Date="1117"
TrackName = ["test","Arlequin", "Eusebins", "Floresla", "Pierrot", "Valse Nobles"]
VideoType = ["","_above", "_side","_short"]
file_name = TrackName[0]+VideoType[0]

video_folder="./sample/videos/"+Date+"/"
os.makedirs(video_folder, exist_ok=True)
json_folder="./sample/json/"+Date+"/"
os.makedirs(json_folder, exist_ok=True)
json_file = json_folder+file_name+".json"
image_folder = "./sample/images/" + Date + "/"
os.makedirs(image_folder, exist_ok=True)

# Initialize the library, if the library is not found, add the library path as an argument
pykinect.initialize_libraries(track_body=True)

# Modify camera configuration
device_config = pykinect.default_configuration
device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1080P
device_config.color_format = pykinect.K4A_IMAGE_FORMAT_COLOR_BGRA32
device_config.camera_fps = pykinect.K4A_FRAMES_PER_SECOND_30
device_config.depth_mode = pykinect.K4A_DEPTH_MODE_WFOV_2X2BINNED
device_config.synchronized_images_only =True
print(device_config)

device = pykinect.start_device(config=device_config)

# Start body tracker
bodyTracker = pykinect.start_body_tracker()
# clamp 調遠方的點雲 rescaling 近處的
# Set the desired depth range for visualization
min_depth = 900
max_depth = 1200 

# 問題不是FOV 而是distortion，depth不能直接拿來用
# 先去照能照到手的問題
# window_size = (1280, 720)
frame_idx = 0
t = time.time()
list_img_RGB=[]
list_normalized_depth=[]
while True:
    capture = device.update()
    ret_rgb, img_RGB = capture.get_color_image()
    ret_depth, img_depth = capture.get_transformed_depth_image()
    
    if not ret_rgb or not ret_depth:
        continue

    normalized_depth = np.clip(img_depth, min_depth, max_depth)
    normalized_depth = (normalized_depth - min_depth) / (max_depth - min_depth) * 255
    normalized_depth = normalized_depth.astype(np.uint8)
    cv2.imshow('MIS Kinect View RGB', img_RGB)

    # Display the color image
    # Save RGB and normalized depth images
    list_img_RGB.append(img_RGB)
    list_normalized_depth.append(normalized_depth)
    # Press q key to stop
    key = cv2.waitKey(1)
    frame_idx += 1
    print(frame_idx)
    if key == ord('q') or key == 27:  # Check for 'q' key or ESC key
        break
    
# Release the resources
print("FPS: ", frame_idx / (time.time() - t))
cv2.destroyAllWindows()
device.close()
for rgbimg in list_img_RGB:
    cv2.imwrite(f"{image_folder}{file_name}_RGB_frame_{frame_idx}.png", rgbimg)
print("RGB Video output and image saving finished!!")
for depthimg in list_img_RGB:
    cv2.imwrite(f"{image_folder}{file_name}_normalized_depth_frame_{frame_idx}.png", depthimg)





Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 2 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: True 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 us. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 us.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64